In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pickle
from pathlib import Path

In [2]:
import torch
import torch.nn.functional as F
from google.protobuf import text_format
from second.utils import simplevis
from second.pytorch.train import build_network
from second.protos import pipeline_pb2
from second.utils import config_tool
from second.pytorch.builder import (box_coder_builder, input_reader_builder,
                                    lr_scheduler_builder, optimizer_builder,
                                    second_builder)
from second.data.preprocess import merge_second_batch, merge_second_batch_multigpu
from second.pytorch.train import _worker_init_fn

from second.pytorch.train import example_convert_to_torch

from second.sphere.conv import init_depth_from_feature, depth_to_3D


## Read Config file

In [3]:
# config_path = "/home/yy/deeplearning/deeplearning/mypackages/second/configs/car.lite.nb.config"
config_path = "/home/gx/GitHub/depconv/second/configs/car.fhd.config"
config = pipeline_pb2.TrainEvalPipelineConfig()
with open(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, config)
input_cfg = config.train_input_reader
model_cfg = config.model.second
# config_tool.change_detection_range(model_cfg, [-50, -50, 50, 50])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


## Build Network, Target Assigner and Voxel Generator

In [4]:
ckpt_path = "/home/gx/GitHub/depconv/second/depconv35/voxelnet-11600.tckpt"
net = build_network(model_cfg).to(device).eval()
net.load_state_dict(torch.load(ckpt_path))
target_assigner = net.target_assigner
voxel_generator = net.voxel_generator

dataset = input_reader_builder.build(
    input_cfg,
    model_cfg,
    training=True,
    voxel_generator=voxel_generator,
    target_assigner=target_assigner,
    multi_gpu=False)

num_gpu=1
collate_fn= merge_second_batch
multi_gpu = False
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=True,
#         num_workers=input_cfg.preprocess.num_workers * num_gpu,
        num_workers=1,
        pin_memory=False,
        collate_fn=collate_fn,
        worker_init_fn=_worker_init_fn,
        drop_last=not multi_gpu)

dense_shape =  [1, 63, 512, 512, 16]
[ 64 512 512]
rpn debug, final_num_filters =  192
rpn debug, num_cls =  2
rpn debug, num_anchor_per_loc =  2
feautres size [1, 64, 64]
res = (1, 64, 64, 1, 2, 7)
feautres size [1, 64, 64]
res = (1, 64, 64, 1, 2, 7)
remain number of infos: 3712


## Generate Anchors

In [5]:
grid_size = voxel_generator.grid_size
feature_map_size = grid_size[:2] // config_tool.get_downsample_factor(model_cfg)
feature_map_size = [*feature_map_size, 1][::-1]

anchors = target_assigner.generate_anchors(feature_map_size)["anchors"]
anchors = torch.tensor(anchors, dtype=torch.float32, device=device)
anchors = anchors.view(1, -1, 7)

feautres size [1, 64, 64]
res = (1, 64, 64, 1, 2, 7)


## Read KITTI infos
you can load your custom point cloud.

In [6]:
info_path = input_cfg.dataset.kitti_info_path
root_path = Path(input_cfg.dataset.kitti_root_path)
with open(info_path, 'rb') as f:
    infos = pickle.load(f)

## Load Point Cloud, Generate Voxels

In [7]:
info = infos[564]
v_path = info["point_cloud"]['velodyne_path']
v_path = str(root_path / v_path)
points = np.fromfile(
    v_path, dtype=np.float32, count=-1).reshape([-1, 4])
# voxels, coords, num_points = voxel_generator.generate(points, max_voxels=90000)
# print(voxels.shape)
# add batch idx to coords
# coords = np.pad(coords, ((0, 0), (1, 0)), mode='constant', constant_values=0)
# voxels = torch.tensor(voxels, dtype=torch.float32, device=device)
# coords = torch.tensor(coords, dtype=torch.int32, device=device)
# num_points = torch.tensor(num_points, dtype=torch.int32, device=device)

## Detection

In [8]:
# example = {
#     "anchors": anchors,
#     "voxels": voxels,
#     "num_points": num_points,
#     "coordinates": coords,
# }
# pred = net(example)[0]
examples = []
loop_i = 0
for example in dataloader:
    loop_i += 1
    if loop_i > 10:
        break
    examples.append(example)
#     print(example)



WORKER 0 seed: 1590158259


/home/gx/GitHub/depconv/second/core/geometry.py:272: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "points_in_convex_polygon_jit" failed type inference due to: Invalid use of Function(<built-in function getitem>) with argument(s) of type(s): (array(float32, 3d, C), Tuple(slice<a:b>, list(int64), slice<a:b>))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    All templates rejected with literals.
In definition 5:
    All templates rejected without literals.
In definition 6:
    All templates rejected with literals.
In definition 7:
    All templates rejected without literals.
In definition 8:
    All templates rejected with literals.
In definition 9:
    All templates rejected without literals.
In definition 

In [9]:
examples[1]['voxels'].shape

(17909, 1, 4)

In [10]:
examples[0]['coordinates'].shape

(17463, 4)

In [11]:
example = examples[2]
example = example_convert_to_torch(example, torch.float32)

# with torch.no_grad():
pred = net(example)


voxel_features shape:  torch.Size([17415, 4])
sparse_shape : [ 64 512 512]
NCDHW =  1 64 3 64 64


In [12]:
voxels = example["voxels"]
num_points = example["num_points"]
coors = example["coordinates"]
num_points = example["num_points"]
preds_dict = net.network_forward(voxels, num_points, coors, batch_size_dev)

NameError: name 'batch_size_dev' is not defined

In [ ]:
pred[3]['box3d_lidar']

In [ ]:
boxes = example['anchors']
boxes
boxes.shape


In [ ]:
boxes[0][1]

In [ ]:
sum(example['points'])

In [ ]:
sum(example['voxels'])

In [ ]:
example['anchors'][0][3027]

In [ ]:
points

In [ ]:
example["points"][0].shape

In [ ]:
res = voxel_generator.generate(
                points, 20000)
res['voxels'].shape

## Simple Vis

In [ ]:
# %config InlineBackend.figure_format = 'retina'


# boxes_lidar = boxes.detach().cpu().numpy()
# boxes_ldar = pred[1]['box3d_lidar']
# vis_voxel_size = [0.1, 0.1, 0.1]
# vis_point_range = [-50, -30, -3, 50, 30, 1]
# vis_point_range = [0, -40, -3, 80, 40, 1]
# bev_map = simplevis.point_to_vis_bev(example["points"], vis_voxel_size, vis_point_range)
# bev_map = simplevis.draw_box_in_bev(bev_map, vis_point_range, boxes_lidar, [0, 255, 0], 2)
# bev_map = simplevis.draw_box_in_bev(bev_map,vis_point_range, boxes_lidar[0][0:8000:80,:].cpu().numpy(), [0,255,0], 2)

In [ ]:
# plt.imshow(bev_map)



In [ ]:
# bev_map = simplevis.kitti_vis(example["points"][0], boxes[0].cpu().numpy())

In [ ]:
# plt.imshow(bev_map)


In [ ]:
example

In [ ]:
example = example_convert_to_torch(example, device=torch.device("cuda"))


voxels = example["voxels"]
num_points = example["num_points"]
coors = example["coordinates"]
batch_anchors = example["anchors"]
# feature = example["feature"]
batch_size_dev = batch_anchors.shape[0]
voxel_features = net.voxel_feature_extractor(voxels, num_points,
                                                      coors)
spatial_features = net.middle_feature_extractor(
            voxel_features, coors, batch_size_dev)
# vfeatures = net.voxel_feature_extractor(voxels, num_points, coors)
# preds_dict = net.network_forward(voxels, num_points, coors, batch_size_dev)
# spatial_features = net.feature_extractor(feature)

In [ ]:
spatial_features.shape

In [ ]:
examples[5]['voxels'].shape

In [ ]:
for i in range(128):
    plt.figure()
    plt.imshow(spatial_features[0][i].detach().cpu().numpy())

In [ ]:

if feature.dim() == 3:
    feature = feature.unsqueeze(0)
with torch.no_grad():
    depth = init_depth_from_feature(feature, 512)

depth = F.max_pool2d(depth, 3, padding=1, stride=1)

In [ ]:
depth.shape

In [ ]:
feature.shape

In [ ]:
# depth_to_3D(feature, depth.long(), D=256)

In [ ]:
plt.imshow(depth[0].detach().cpu().numpy())

In [ ]:
depth[0].detach().cpu().shape

In [ ]:
# depth[depth == 0 ] = 256

In [ ]:
def depth_bev(depth, R =500):
    """
    convert depth to bev
        depth: [H,W]
        to, 
    """
    H, W = depth.shape

    ret = np.zeros([R, W])
    for r in range(H):
        for i in range(W):
            d = int(depth[r][i])
            if d >= R:
                d = R-1
            ret[d][i] = 1
    return ret.T

plt.imshow(depth_bev(depth[0].detach().cpu().numpy()))

In [ ]:
depth1 = F.max_pool2d(depth, (2,1), padding=0, stride=(1,2)) / 2 
plt.imshow(depth_bev(depth1[0].detach().cpu().numpy(), 256))


In [ ]:
depth1

In [ ]:
depth2 = F.max_pool2d(depth1, (2,1), padding=0, stride=(1,2))/2
plt.imshow(depth_bev(depth2[0].detach().cpu().numpy(),  R=128))
print(depth2.shape)

In [ ]:
depth3 = F.avg_pool2d(depth2, (2,1), padding=0, stride=(1,2))/2
plt.imshow(depth_bev(depth3[0].detach().cpu().numpy(),  R=64))
print(depth3.shape)

In [ ]:
depth3.shape

In [ ]:
depth4 = F.max_pool2d(depth3, (2,1), padding=0, stride=(1,2)) /2 
plt.imshow(depth_bev(depth4[0].detach().cpu().numpy(), 32))
print(depth4.shape)

In [ ]:
depth5 = F.max_pool2d(depth4, (2,1), padding=(0,0), stride=(1,2)) /2 
plt.imshow(depth_bev(depth5[0].detach().cpu().numpy(), 16))
print(depth5.shape)

In [ ]:
bev = torch.tensor(depth_bev(depth[0].detach().cpu().numpy())).unsqueeze(0)
bev1 = F.max_pool2d(bev, 2, stride=2)
plt.imshow(bev1[0])

In [ ]:
bev2 = F.max_pool2d(bev1, 2, stride=2)
plt.imshow(bev2[0])

In [ ]:
bev3 = F.max_pool2d(bev2, 2, stride=2)
plt.imshow(bev3[0])

In [ ]:
bev4 = F.max_pool2d(bev3, 2, stride=2)
plt.imshow(bev4[0])

In [ ]:
bev5 = F.max_pool2d(bev4, 2, stride=2)
plt.imshow(bev5[0])

In [ ]:

# x = net.feature_extractor.conv1(feature, depth)
# x = net.feature_extractor.bn1(x)
# x = F.relu(x)

In [ ]:
# x.shape

In [ ]:
# plt.imshow(x[0][5].detach().cpu().numpy())

In [ ]:
depth_ = F.avg_pool2d(depth.float(), 2, padding=1, stride=(2,2)).long()

In [ ]:
plt.imshow(depth_[0].detach().cpu().numpy())

In [ ]:
for i in car_anchor_i:
    x = i % 64
    y = i // 64
    print(x,y)
    

In [ ]:
# example = examples[5]
# example = example_convert_to_torch(example, torch.float32)
feature = example['feature']
if feature.dim() == 3:
    feature = feature.unsqueeze(0)
with torch.no_grad():
    depth = init_depth_from_feature(feature, 512)
    
plt.figure(figsize = (25,25))
plt.imshow(depth[0].detach().cpu().numpy())

In [ ]:
example['metadata']

In [ ]:



car_anchor_i = []
for i, t in enumerate(example['reg_targets'][0]):
    if t.sum() > 0:
        print(i, t)
        car_anchor_i.append(i)
# car_anchor_i

car_anchors = example['anchors'][0][car_anchor_i]
car_anchors

vis_voxel_size = [0.01, 0.1, 0.1]
vis_point_range = [0, -30, -3, 6, 30, 1]
bev_map = simplevis.point_to_vis_bev(np.stack((np.log(r),points[:,1],points[:,2]), axis=1), vis_voxel_size, vis_point_range)

# bev_map = simplevis.kitti_vis(example['points'][0],
#                               car_anchors.cpu().numpy()  )
plt.figure(figsize = (10,10))
plt.imshow(bev_map)

In [ ]:

pnts = points
pnts[3] = np.log(pnts[3])

In [ ]:
plt.plot(points[:,x])

In [ ]:
r = np.sqrt(points[:,0]**2 + points[:,1]**2 + points[:,2]**2)

In [ ]:
plt.plot(r)

In [ ]:
plt.plot(np.log(r))

In [ ]:
np.stack((points[:,0],points[:,1], r), axis=1).shape

In [ ]:
points[:,1].shape